In [1]:
import numpy as np
import os
import xml.dom.minidom
import xml.etree.cElementTree as ET
import copy
from PIL import Image
from datetime import datetime

In [25]:
class Place():
    place_counter = -1
    """
    Eingangsdaten:
    Klassenpfad, Ausgangsbildgrösse in px, Anzahl Bilder Channel
    
    Konstruktor genieriert ein dictonary, welches wie folgt aussieht:
    
    {Pfad_Klassen:{Klassenname_1:{0:Bildname_1, 1:Bildname1}}
                 :{Klassenname_2:{0:Bildname_1, 1:Bildname2}}
    }
    
    2 Varianten wie die einzelen Zeichen platziert werden können:
    
    1. rdm Verteilung
    2. ....
    
    """
    
    def __init__(self, path, xml_path, segemented_image_path, size, depth, iou_threshold):
        self._path = path
        self._xml_path = xml_path
        self._segemented_image_path = segemented_image_path
        self._size = size
        self._depth= depth
        self._iou_threshold = iou_threshold
        self._dictonary = {'path':{'folder_name':{0:'image_name'}}}
        self._segmented_image = np.zeros((self._size, self._size, self._depth))
        self._bounding_boxes = np.array([0,0,0,0])
        self._body_name = ''
        
        path_1, dirs_1, files_1 = next(os.walk(self._path))
        self._dictonary = {path_1:{}}
        for folder in dirs_1:
            self._dictonary[path_1][folder] = {}
#             print(folder)
            path_2, dirs_2, files_2 = next(os.walk(path_1 + folder))
            for i, files in zip(range(len(files_2)), files_2):
                self._dictonary[path_1][folder][i] = {files}
#                 print(files) 


    def get_head(self,place_image_path,width,height,depth):
        now = datetime.now()
        self._date_time = now.strftime("%Y%m%d_%H%M%S")
        filename = self._date_time
        slice_position_r = place_image_path.rfind('/')
        slice_position_l = place_image_path[slice_position_r+1:]
        folder = place_image_path[slice_position_r+1:slice_position_l]        
        path = place_image_path
        string_head =   '<annotation>'\
                        '<folder>'+str(folder)+'</folder>'\
                        '<filename>'+str(filename)+'</filename>'\
                        '<path>'+str(path)+'</path>'\
                        '<source>'\
                        '<database>Unknown</database>'\
                        '</source>'\
                        '<size>'\
                            '<width>'+str(self._size)+'</width>'\
                            '<height>'+str(self._size)+'</height>'\
                            '<depth>'+str(self._depth)+'</depth>'\
                        '</size>'\
                        '<segmented>0</segmented>'\
                        '</annotation>'
        return string_head

    
    def get_body(self,image_name,xmin,ymin,xmax,ymax):
        slice_position = image_name.find('_')
        name = image_name[:slice_position]
        string_object = '<object>'\
                        '<name>'+str(name)+'</name>'\
                        '<pose>Unspecified</pose>'\
                        '<truncated>0</truncated>'\
                        '<difficult>0</difficult>'\
                        '<bndbox>'\
                            '<xmin>'+str(xmin)+'</xmin>'\
                            '<ymin>'+str(ymin)+'</ymin>'\
                            '<xmax>'+str(xmax)+'</xmax>'\
                            '<ymax>'+str(ymax)+'</ymax>'\
                            '</bndbox>'\
                        '</object>'
        return string_object

    
    def iou_check(self,x_rdm_min,y_rdm_min,x_rdm_max,y_rdm_max,iou_threshold,place_image_path):
        print('iou_check')
        if self._segmented_image.all()==0.0:
            self._bounding_boxes[0] = x_rdm_max
            self._bounding_boxes[1] = y_rdm_max
            self._bounding_boxes[2] = x_rdm_min
            self._bounding_boxes[3] = y_rdm_min
            tmp_counter = 0
            result = True
        else:
            bounding_box[0] = x_rdm_max
            bounding_box[1] = y_rdm_max
            bounding_box[2] = x_rdm_min
            bounding_box[3] = y_rdm_min
            length = len(self._bounding_boxes)/4
            for i in range(length):
                tmp_start = 0+(4*i)
                tmp_end = 3+(4*i)
                tmp_bounding_boxes = self._bounding_boxes[tmp_start:tmp_end]
                iou = bb_intersection_over_union(bounding_box, tmp_bounding_boxes)
                if iou <= iou_threshold:
                    np.append(self._bounding_boxes,bounding_boxes)
                    tmp_counter = 0
                    result = True
                else:
                    if tmp_counter == 3:
                        result = False
                    else:
                        tmp_counter += 1
                        place(self, place_image_path)         
        return result
    
    
    def place(self, place_image_path):
        with Image.open(place_image_path) as image:
            img = np.array(image)
            y,x,z = img.shape
#             print(y,x,z)
            
            if Place.place_counter == 0:
                
                x_max = self._size - x - 5
                y_max = self._size - y - 5
                x_rdm_min = np.random.randint(5,x_max)
                y_rdm_min = np.random.randint(5,y_max)
                x_rdm_max = x_rdm_min + x
                y_rdm_max = y_rdm_min + y
                result = self.iou_check(x_rdm_min,y_rdm_min,x_rdm_max,y_rdm_max,self._iou_threshold,place_image_path)
#                 print(x_rdm_min,y_rdm_min,x_rdm_max,y_rdm_max)
                y1,x1,z1 = self._segmented_image.shape
#                 print(y1,x1,z1)
#                 print(result)
                if result == True:
                    self._segmented_image[y_rdm_min:y_rdm_max,x_rdm_min:x_rdm_max,:] = img
                    return self._segmented_image, x_rdm_min,y_rdm_min,x_rdm_max,y_rdm_max, True 
                    
                else: 
                    return self._segmented_image, x_rdm_min,y_rdm_min,x_rdm_max,y_rdm_max, False
                    
          
        print(result)
        
    def create_xml_segmented_image(self, string):
        
        dom = xml.dom.minidom.parseString(string)
        pretty_xml_as_string_body = dom.toprettyxml()

        filepath = self._xml_path + '/' + str(date_time) + '.xml'
        with open(filepath, 'w+') as file:
            file.write(pretty_xml_as_string_body)
        
        im = Image.fromarray(self._segmented_image)
        im.save('self._segemented_image_path'+'/'+'date_time'+'.bmp')
        self._segmented_image = np.zeros((self._size, self._size, self._depth))
            
        
    def create_xml_body(self,place_image_path,x_rdm_min,y_rdm_min,x_rdm_max,y_rdm_max):

            slice_position = place_image_path.find('_')
            object_name = place_image_path[:slice_position]
            xml_name_tmp = self.get_body(place_image_path,x_rdm_min-1,y_rdm_min-1,x_rdm_max+1,y_rdm_max+1)
            self._body_name = self._body_name + xml_name_tmp

              
    def create_xml(self,place_image_path):
#         name = str(Place.placeIt_counter) + 
        xml_head = get_body(dict_folder,x_rdm_min-1,y_rdm_min-1,x_rdm_max+1,y_rdm_max+1)
        slice_position = xml_head.find('</annotation>')
        complete_xml = xml_head[:slice_position] + self._body_name + xml_head[slice_position:]
        
        create_xml_segmented_image(full_xml)
        self._body_name = ''     
       
    
    def delete_dict(self, dict_folder,number_dict_images): 
        del self._dictonary[self._path][dict_folder][number_dict_images]
        if len(list(self._dictonary[self._path][dict_folder]))== 0:
            del self._dictonary[self._path][dict_folder]


    def rdm(self):
        
        ### Zufälliger Ordner von Zeichen z.B. A oder B ###
        dict_folders = list(self._dictonary[self._path].keys())
        print(dict_folders)
        if len(dict_folders) == 0:
            number_dict_folders = 0
        else:
            number_dict_folders = np.random.randint(len(dict_folders))
        print(number_dict_folders)
        
        if number_dict_folders == 0:
            rdm_folder = 0
        else:
            rdm_folder = np.random.randint(number_dict_folders)
        print(rdm_folder)
        dict_folder = dict_folders[rdm_folder]
        print('dict_folder: ',dict_folder)
        
        ### Zufälliges Bild von oben bestimmten Zeichen ###
        dict_images = list(self._dictonary[self._path][dict_folder].keys())
        print('dict_images: ',dict_images)
        if len(dict_images) == 0:
            number_dict_images = 0
        else:
            number_dict_images = np.random.randint(len(dict_images))
        print(number_dict_images)
        
        if number_dict_images == 0:
            rdm_image = 0
        else:
            rdm_image = np.random.randint(number_dict_images)
            
        print(rdm_image)
        dict_image_number = dict_images[rdm_image]
        dict_image = str(self._dictonary[self._path][dict_folder][dict_image_number])
        print(dict_image)
        
        r_tmp = dict_image.find('}')
        dict_image = dict_image[2:r_tmp-1]
        print(dict_image)
        
        ### Bild Pfad des Zeichens ###
        place_image_path = self._path + dict_folder + '/' +  dict_image
        Place.place_counter +=1
        
        
        self._segmented_image, x_rdm_min,y_rdm_min,x_rdm_max,y_rdm_max, placed = self.place(place_image_path)
        print(self._segmented_image, x_rdm_min,y_rdm_min,x_rdm_max,y_rdm_max, placed)
        ################################## Hier weiter machen, bis hier hin kam der code
        
        
        if placed == True:
            self.create_xml_body(place_image_path, x_rdm_min,y_rdm_min,x_rdm_max,y_rdm_max)    
            self.delete_dict(dict_folder,number_dict_images)
        else:
            self.create_xml(place_image_path,x_rdm_min,y_rdm_min,x_rdm_max,y_rdm_max,place_image_path)  
            rdm()
    
    
    def bb_intersection_over_union(self, boxA, boxB):
        xA = max(boxA[0], boxB[0])
        yA = max(boxA[1], boxB[1])
        xB = min(boxA[2], boxB[2])
        yB = min(boxA[3], boxB[3])
        
        # compute the area of intersection rectangle
        interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
        # compute the area of both the prediction and ground-truth
        # rectangles
        boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
        boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
        # compute the intersection over union by taking the intersection
        # area and dividing it by the sum of prediction + ground-truth
        # areas - the interesection area
        iou = interArea / float(boxAArea + boxBArea - interArea)
        # return the intersection over union value
        return iou    
    
        
        
#     def create_xml(self):    
#     def segmented_image(self):

In [26]:
obj33 = Place('C:/Users/User/Desktop/Test/GeneratedData/TailoredImages/','C:/Users/User/Desktop/Test/GeneratedData/XML_Files/','C:/Users/User/Desktop/Test/GeneratedData/SegmentedImages',512,3,0.3)

In [27]:
obj33.rdm()

['A_upperCase', 'B_upperCase']
0
0
dict_folder:  A_upperCase
dict_images:  [0, 1, 2]
1
0
{'A_upperCase_arial_0_tailored.BMP'}
A_upperCase_arial_0_tailored.BMP
152 93 3
iou_check
213 136 306 288
512 512 3
True
[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]] 213 136 306 288 True


In [6]:
obj33._dictonary['C:/Users/User/Desktop/Test/GeneratedData/TailoredImages/']['A_upperCase'][0]
len()

{'A_upperCase_arial_0_tailored.BMP'}

In [ ]:
b = 'hallo'
a = c + b
print(a)

In [5]:
for classes in list(obj33._dictonary['C:/Users/User/Desktop/Test/GeneratedData/TailoredImages/']):
    class_1 = list(obj33._dictonary['C:/Users/User/Desktop/Test/GeneratedData/TailoredImages/'])
    number_images = len(list(list(obj33._dictonary['C:/Users/User/Desktop/Test/GeneratedData/TailoredImages/'][class_1[0]])))
    if number_images != len(list(list(obj33._dictonary['C:/Users/User/Desktop/Test/GeneratedData/TailoredImages/'][classes]))):
        print('Ungleichgewicht: Nicht gleichviele Bilder von jeder Klasse vorhanden, siehe Klasse: ' + classes)
        break
#     print('alles i.O')
#     print(class_1[0])
#     print(number_images)
#     print(classes)
# # len(list(obj33._dictonary['C:/Users/User/Desktop/Test/GeneratedData/TailoredImages/']))
# list(obj33._dictonary['C:/Users/User/Desktop/Test/GeneratedData/TailoredImages/'])

Ungleichgewicht: Nicht gleichviele Bilder von jeder Klasse vorhanden, siehe KlasseB_upperCase


In [5]:
print(list(obj33._dictonary))

['C:/Users/User/Desktop/Test/GeneratedData/TailoredImages/']


In [7]:
print(len(list(obj33._dictonary['C:/Users/User/Desktop/Test/GeneratedData/TailoredImages/']['A_upperCase'])))
del obj33._dictonary['C:/Users/User/Desktop/Test/GeneratedData/TailoredImages/']['A_upperCase'][2]

2


KeyError: 2

In [19]:
def get_head():
    string_head =   '<annotation>'\
                    '<folder>'+str('hallo')+'</folder>'\
                    '<filename>'+str('filename')+'</filename>'\
                    '<path>'+str('path')+'</path>'\
                    '<source>'\
                    '<database>Unknown</database>'\
                    '</source>'\
                    '<size>'\
                        '<width>'+str('self._size')+'</width>'\
                        '<height>'+str('self._size')+'</height>'\
                        '<depth>'+str('self._depth')+'</depth>'\
                    '</size>'\
                    '<segmented>0</segmented>'\
                    '</annotation>'
    return string_head

a = get_head()
a.find('</annotation>')
print(a[240:])
print(a[:240])

</annotation>
<annotation><folder>hallo</folder><filename>filename</filename><path>path</path><source><database>Unknown</database></source><size><width>self._size</width><height>self._size</height><depth>self._depth</depth></size><segmented>0</segmented>


In [22]:
dom = xml.dom.minidom.parseString(a)
pretty_xml_as_string_body = dom.toprettyxml()

In [39]:
a = {'C':{0:{'a':0}}}
print(len(a['C'][0]))
del a['C'][0]
print(len(a))
print(len(a['C']))

1
1
0


In [2]:
a = 'C:/Users/User/Desktop/Ablaufdatum/ablaufdatum_selber/all/20180528_200104.jpg'

In [10]:
b = a.rfind('/')
print(a[b+1:])
c = a[:b].rfind('/')
print(c)
print(a[c+1:b])

20180528_200104.jpg
52
all
